<img align="right" width="200" height="200" src="https://static.tildacdn.com/tild6236-6337-4339-b337-313363643735/new_logo.png">

# Введение в Spark: RDD API
**Сергей Гришаев**  
serg.grishaev@gmail.com  

## На этом занятии
+ Общие сведения
+ Область применения
+ Архитектура приложений
+ Базовые функции RDD API
+ Рair RDD функции
+ Работа с данными

## Общие сведения

**Apache Spark** - это:
+ Платформа для построения распределнных приложений обработки данных
+ Эволюция Hadoop MapReduce
+ Библиотека под Python/Scala
+ Один из самых популярных проектов в области обработки больших данных


## Область применения
- Распределенная обработка больших данных
- Построение ETL пайплайнов
- Работа со структурированными данными (SQL)
- Разработка стриминг приложений

## Архитектура приложения

+ **Driver** (aka Master):
  - предоставляет API через SparkSession и SparkContext
  - выполняет ваш код - python файл или скомпилированный .jar
  - контролирует выполнение задачи

+ **Workers** (aka Executors or Slaves):
  - обрабатывают данные
  - каждый Worker работает со своим сегментом данных - **Partition**
  - не выполняются ваш код напрямую
  - получают задачи от Driver
+ **Cluster Manager** (YARN/Mesos):
  - отвечает за аллокацию контейнеров, выполняющих код драйвера и воркеров, на кластере
  - квотирует ресурсы между пользователями
  - контролирует состояние контейнеров
 
<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-06-05%2013.07.01.jpeg">

## Resilient Distributed Dataset

**RDD** aka Resilient Distributed Dataset - самая базовая и самая низкоуровневая структура в Spark, доступная разработчику. Представляет собой типизированную неизменяемую неупорядоченную партиционированную коллекцию данных, распределенную по узлам кластера

In [1]:
val cities: Vector[String] = Vector("Moscow", "Paris", "Madrid", "London", "New York")
println(s"The Vector has ${cities.length} elements, the first one is ${cities.head}}")

The Vector has 5 elements, the first one is Moscow}


cities = Vector(Moscow, Paris, Madrid, London, New York)


Vector(Moscow, Paris, Madrid, London, New York)

In [21]:
val cities = Vector("Moscow", "Paris", "Madrid", "London", "New York", "Cape Town")


cities = Vector(Moscow, Paris, Madrid, London, New York, Cape Town)


Vector(Moscow, Paris, Madrid, London, New York, Cape Town)

In [2]:
var cities2: Vector[String] = Vector("Moscow", "Paris", "Madrid", "London", "New York")

cities2 = Vector(Moscow, Paris, Madrid, London, New York)


Vector(Moscow, Paris, Madrid, London, New York)

In [4]:
cities2 = Vector("Moscow", "Paris", "Madrid", "London", "New York", "Test city")

cities2 = Vector(Moscow, Paris, Madrid, London, New York, Test city)


Vector(Moscow, Paris, Madrid, London, New York, Test city)

In [5]:
cities2 = Vector()

cities2 = Vector()


Vector()

In [8]:
val newCities = cities :+ "Madrid"

newCities = Vector(Moscow, Paris, Madrid, London, New York, Madrid)


Vector(Moscow, Paris, Madrid, London, New York, Madrid)

In [7]:
cities

Vector(Moscow, Paris, Madrid, London, New York)

In [9]:
newCities

Vector(Moscow, Paris, Madrid, London, New York, Madrid)

In [10]:
def toUpper(x: String): String = { 
    x.toUpperCase
}

toUpper: (x: String)String


In [14]:
val toUpper2 = (x: String) => {
    x.toUpperCase
}

toUpper2 = > String = <function1>


> String = <function1>

In [13]:
toUpper("foo")

FOO

In [12]:
toUpper2("foo")

FOO

In [15]:
cities
    .sorted
    .reverse
    .map(x => x.toUpperCase)

Vector(PARIS, NEW YORK, MOSCOW, MADRID, LONDON)

In [16]:
cities
    .sorted
    .reverse
    .map(toUpper)

Vector(PARIS, NEW YORK, MOSCOW, MADRID, LONDON)

In [18]:
val cities = Vector()

cities = Vector()


Vector()

In [26]:
cities(1)

Paris

In [27]:
cities.apply(1)

Paris

In [28]:
// val spark = SparkSession.builder.getOrCreate()
spark

Waiting for a Spark session to start...

org.apache.spark.sql.SparkSession@7d7eac85

In [29]:
sc

Waiting for a Spark session to start...

org.apache.spark.SparkContext@50060ba7

In [30]:
spark

org.apache.spark.sql.SparkSession@7d7eac85

In [31]:
import org.apache.spark.rdd.RDD

val rdd: RDD[String] = sc.parallelize(cities)
println(s"The RDD has ${rdd.count} elements, the first one is ${rdd.first}")
println(f"The RDD has ${rdd.count} elements, the first one is ${rdd.first}")

The RDD has 6 elements, the first one is Moscow
The RDD has 6 elements, the first one is Moscow


rdd = ParallelCollectionRDD[0] at parallelize at <console>:31


ParallelCollectionRDD[0] at parallelize at <console>:31

RDD может быть создана из:
- локальной коллекции на драйвере
- файла (локального или на распределенной файловой системе, например HDFS)
- базы данных

### Операции с RDD
1. Трансформации (e.g. map, filter)
2. Действия (e.g. reduce, collect, count, foreach)

**Трансформации** (Transormations):
- всегда превращают один RDD в новый RDD
- всегда являются ленивыми - создают граф вычислений, но не запускают их
- иногда (часто) неявно требуют перемешивания данных между воркерами - **Shuffle**

In [33]:
// Трансформация map: не запускает вычислений, не изменяет изначальный RDD
val upperRdd: RDD[String] = rdd.map { city => city.toUpperCase }


// Метод take возвращает N первых элементов RDD
val upperVec: Array[String] = upperRdd.take(3)
val oldVec: Array[String] = rdd.take(3)

// метод mkString позволяет сделать из любой локальной коллекции строку
println(s"""New RDD: ${upperVec.mkString(", ")}""")
println(s"""Old RDD: ${oldVec.mkString(", ")}""")

New RDD: MOSCOW, PARIS, MADRID
Old RDD: Moscow, Paris, Madrid


upperRdd = MapPartitionsRDD[2] at map at <console>:33
upperVec = Array(MOSCOW, PARIS, MADRID)
oldVec = Array(Moscow, Paris, Madrid)


Array(Moscow, Paris, Madrid)

In [46]:
val oldVec = Array("foo")

oldVec = Array(foo)


Array(foo)

In [44]:
oldVec = Array("foo2")

oldVec = Array(foo2)


Array(foo2)

In [42]:
oldVec(0) = "foo123"

In [43]:
oldVec(0)

foo123

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2019.15.06.jpeg">

**Действия** (Actions):
- выполняют действие над RDD
- запускают вычисления

In [47]:
// Действие reduce применяет функцию f к промежуточному результату 
// от предыдущей итерации со следующим элементом коллекции
val count = rdd.map( x => x.length ).reduce { (x,y) => x + y }
println(s"The RDD contains ${count} letters")

The RDD contains 40 letters


count = 40


40

In [49]:
rdd.getClass.getCanonicalName

org.apache.spark.rdd.ParallelCollectionRDD

In [48]:
count.getClass.getCanonicalName

int

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2019.22.10.jpeg">

### Примеры операций с RDD

Фильтрация RDD

In [51]:
val startsWithM = upperRdd.filter( x => x.startsWith("M") )
println(s"""The following city names starts with M: ${startsWithM.take(5).mkString(", ")}""")

The following city names starts with M: MOSCOW, MADRID


startsWithM = MapPartitionsRDD[5] at filter at <console>:30


MapPartitionsRDD[5] at filter at <console>:30

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2019.33.22.jpeg">

Подсчет количества элементов в RDD

In [52]:
val countM: Long = startsWithM.count()
println(s"The RDD contains $countM elements")

The RDD contains 2 elements


countM = 2


2

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2019.45.03.jpeg">

Передача ВСЕХ элементов RDD на драйвер

In [53]:
val localArray: Array[String] = startsWithM.collect()
val containsMoscow: Boolean = localArray.contains("MOSCOW")
println(s"The array contains MOSCOW: $containsMoscow")

The array contains MOSCOW: true


localArray = Array(MOSCOW, MADRID)
containsMoscow = true


true

In [65]:
startsWithM.getClass.getDeclaredMethods.mkString("\n")

public scala.Option org.apache.spark.rdd.MapPartitionsRDD.partitioner()
public boolean org.apache.spark.rdd.MapPartitionsRDD.isBarrier_()
private boolean org.apache.spark.rdd.MapPartitionsRDD.isBarrier_$lzycompute()
public scala.Enumeration$Value org.apache.spark.rdd.MapPartitionsRDD.getOutputDeterministicLevel()
public org.apache.spark.Partition[] org.apache.spark.rdd.MapPartitionsRDD.getPartitions()
public void org.apache.spark.rdd.MapPartitionsRDD.clearDependencies()
public static boolean org.apache.spark.rdd.MapPartitionsRDD.$lessinit$greater$default$4()
public static boolean org.apache.spark.rdd.MapPartitionsRDD.$lessinit$greater$default$5()
public static boolean org.apache.spark.rdd.MapPartitionsRDD.$lessinit$greater$default$3()
public void org.apache.spark.rdd.Ma...


Передача N элементов по сети на драйвер

In [60]:
val twoElements: Array[String] = startsWithM.take(1)
println(s"""Two elements of the RDD are: ${twoElements.mkString(", ")}""")

Two elements of the RDD are: MOSCOW


twoElements = Array(MOSCOW)


Array(MOSCOW)

Сортировка и выборка из N первых элементов

In [66]:
val twoElementsSorted: Array[String] = startsWithM.takeOrdered(2)
println(s"""Two elements of the RDD are: ${twoElementsSorted.mkString(", ")}""")

Two elements of the RDD are: MADRID, MOSCOW


twoElementsSorted = Array(MADRID, MOSCOW)


Array(MADRID, MOSCOW)

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2020.40.09.jpeg">

Спрямление вложенных коллекций

In [68]:
val mappedRdd: RDD[Vector[Char]] = rdd.map(x => x.toVector)
mappedRdd.take(2).foreach(println)
println("####")

val flatMappedRdd = rdd.flatMap( x => x.toLowerCase )
flatMappedRdd.take(20).foreach(println)

Vector(M, o, s, c, o, w)
Vector(P, a, r, i, s)
####
m
o
s
c
o
w
p
a
r
i
s
m
a
d
r
i
d
l
o
n


mappedRdd = MapPartitionsRDD[9] at map at <console>:31
flatMappedRdd = MapPartitionsRDD[10] at flatMap at <console>:35


MapPartitionsRDD[10] at flatMap at <console>:35

In [69]:
mappedRdd.take(2).foreach(x => println(x))

Vector(M, o, s, c, o, w)
Vector(P, a, r, i, s)


In [70]:
mappedRdd.take(2).foreach(println)

Vector(M, o, s, c, o, w)
Vector(P, a, r, i, s)


In [71]:
mappedRdd.take(2).foreach( println(_) )

Vector(M, o, s, c, o, w)
Vector(P, a, r, i, s)


In [72]:
rdd.flatMap(_.toLowerCase).collect

Array(m, o, s, c, o, w, p, a, r, i, s, m, a, d, r, i, d, l, o, n, d, o, n, n, e, w,  , y, o, r, k, c, a, p, e,  , t, o, w, n)

Удаление дубликатов

In [73]:
val uniqueLetters: String = 
    flatMappedRdd
        .distinct
        .filter(x => x != ' ')
        .collect
        .sorted
        .mkString(" ")

println(s"Letters in the RDD are: ${uniqueLetters}")

Letters in the RDD are: a c d e i k l m n o p r s t w y


uniqueLetters = a c d e i k l m n o p r s t w y


a c d e i k l m n o p r s t w y

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2020.42.42.jpeg">

### Выводы
- RDD - это неизменяемый распределенный набор данных
- Трансформации (map, filter, flatMap) создают новый RDD из существующего и не изменяют существующий
- Любые трансформации являются ленивыми и не запускают вычислений
- Действия (count, reduce, collect, take) запускают вычисления

### Полезные ссылки:
- [RDD API Reference](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD)
- [RDD Programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
- [Scala 2.11.12 API](https://www.scala-lang.org/files/archive/api/2.11.12)

## PairRDD функции

Во всех вышестоящих экспериментах мы создавали RDD, состоящие из элементов базовых типов - числовых, срок и символов. На самом деле RDD не имеют как таковых органичений на тип элементов. Ими могут выступать коллекции, кейс 
классы, кортежи и т.д.

**PairRDD** - расширенный класс функций, доступных для RDD, где элементы - это кортеж (key, value)

In [74]:
val foo: (Int, String) = (1, "foo")

foo = (1,foo)


(1,foo)

In [75]:
foo._1

1

In [76]:
foo._2

foo

In [77]:
val (left: Int, right: String) = foo

left = 1
right = foo


foo

In [78]:
val bar = List(1,2,3)

bar = List(1, 2, 3)


List(1, 2, 3)

In [79]:
val List(first, second, third) = bar

first = 1
second = 2
third = 3


3

In [80]:
val pairRdd: RDD[(Char, Int)] = rdd.flatMap( x => x.toLowerCase ).map( x => (x, 1))
pairRdd.take(4).foreach(println)

(m,1)
(o,1)
(s,1)
(c,1)


pairRdd = MapPartitionsRDD[17] at map at <console>:28


MapPartitionsRDD[17] at map at <console>:28

In [81]:
pairRdd.map (x => x._1).collect

Array(m, o, s, c, o, w, p, a, r, i, s, m, a, d, r, i, d, l, o, n, d, o, n, n, e, w,  , y, o, r, k, c, a, p, e,  , t, o, w, n)

In [88]:
pairRdd.map { case (_, v) => v }.collect

Array(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

In [100]:
val iter: Iterator[Any] = foo.productIterator

iter = non-empty iterator


non-empty iterator

In [101]:
while (iter.hasNext) { 
    val next = iter.next
    println(next)
}

1
foo


In [98]:
iter.toList.head

1

In [ ]:
Iterator[T]

countByKey подсчитывает количество кортежей по каждому ключу и возвращает локальный Map

In [102]:
val letterCount: scala.collection.Map[Char,Long] = pairRdd.countByKey

letterCount = Map(e -> 2, s -> 2, n -> 4, y -> 1, t -> 1, a -> 3, m -> 2, i -> 2,   -> 2, l -> 1, p -> 2, c -> 2, r -> 3, w -> 3, k -> 1, o -> 6, d -> 3)


Map(e -> 2, s -> 2, n -> 4, y -> 1, t -> 1, a -> 3, m -> 2, i -> 2,   -> 2, l -> 1, p -> 2, c -> 2, r -> 3, w -> 3, k -> 1, o -> 6, d -> 3)

In [103]:
letterCount('e')

2

In [104]:
letterCount('b')

Name: java.util.NoSuchElementException
Message: key not found: b
StackTrace:   at scala.collection.MapLike$class.default(MapLike.scala:228)
  at scala.collection.AbstractMap.default(Map.scala:59)
  at scala.collection.MapLike$class.apply(MapLike.scala:141)
  at scala.collection.AbstractMap.apply(Map.scala:59)

In [105]:
val bar: Option[Long] = letterCount.get('b')

bar = None


lastException: Throwable = null


None

In [106]:
val foo: Option[Long] = letterCount.get('e')

foo = Some(2)


Some(2)

In [107]:
val x = bar match { 
    case Some(v) => v
}

Name: scala.MatchError
Message: None (of class scala.None$)
StackTrace: It would fail on the following input: None
       val x = bar match {
               ^
scala.MatchError: None (of class scala.None$)

In [109]:
val y = bar match { 
    case Some(v) => v
    case None => -1 
}

y = 2


2

In [110]:
val foo: Long = 1

foo = 1


1

In [111]:
val foo: Option[Long] = Some(1)

foo = Some(1)


Some(1)

In [113]:
foo.map(x => x + 1).filter( x => x == 2 )

Some(2)

In [119]:
List(1,2,3).head

1

In [114]:
val foo: Option[List[Int]] = Option(List())

foo = Some(List())


Some(List())

In [115]:
val elem: Option[Int] = foo.flatMap(x => x.headOption)

elem = None


None

In [120]:
val foo: Option[Long] = None

foo = None


None

In [121]:
val test1: Int = null

Name: Unknown Error
Message: <console>:26: error: an expression of type Null is ineligible for implicit conversion
       val test1: Int = null
                        ^

StackTrace: 

In [122]:
val foo: java.lang.Integer = 1

foo = 1


1

In [125]:
foo.isInstanceOf[Int]

true

In [126]:
val foo: Int = null

Name: Unknown Error
Message: <console>:26: error: an expression of type Null is ineligible for implicit conversion
       val foo: Int = null
                      ^

StackTrace: 

In [127]:
val foo: java.lang.Integer = null

foo: Integer = null


In [129]:
foo.isInstanceOf[Int]

false

In [128]:
foo

res302: Integer = null


In [132]:
val myList = letterCount.values.toList.distinct.sorted.take(3)
myList match { 
    case 1 :: x => println(s" $x : hooray!")
    case _ => println("hello!")
}

 List(2, 3) : hooray!


myList = List(1, 2, 3)


List(1, 2, 3)

In [133]:
Map( (1, "foo"), (2, "bar"))

Map(1 -> foo, 2 -> bar)

In [134]:
1 -> "foo"

(1,foo)

In [135]:
Map( 1 -> "foo", 2 -> "bar").contains(1)

true

reduceByKey работает аналогично обычному reduce, но промежуточный итог накапливается по каждому ключу независимо

In [137]:
val letterCount: RDD[(Char, Int)] = pairRdd.reduceByKey { (x,y) => x + y }
println(letterCount.take(3).mkString(" "))

(d,3) (p,2) (t,1)


letterCount = ShuffledRDD[27] at reduceByKey at <console>:32


ShuffledRDD[27] at reduceByKey at <console>:32

<img style="float: left;" src="https://raw.githubusercontent.com/tenkeiu8/spark-examples/master/images/photo_2021-09-19%2020.57.17.jpeg">

Join позволяет соединить два RDD по ключу. Поддерживаются join, leftOuterJoin и fullOuterJoin

In [138]:
val favouriteLetters: Vector[Char] = Vector('a', 'd', 'o')
val favLetRdd = sc.parallelize(favouriteLetters).map(x => (x,1))


val joined: RDD[(Char, (Int, Option[Int]))] = letterCount.leftOuterJoin(favLetRdd)
joined.collect.foreach { j => 
    val (letter, (leftCount, rightCount)) = j
    rightCount match {
        case Some(v) => println(s"The letter $letter is my favourite and it appears in the RDD $leftCount times")
        case None => println(s"The letter $letter is not my favourite!")
    }
}

The letter d is my favourite and it appears in the RDD 3 times
The letter p is not my favourite!
The letter t is not my favourite!
The letter   is not my favourite!
The letter n is not my favourite!
The letter r is not my favourite!
The letter l is not my favourite!
The letter w is not my favourite!
The letter s is not my favourite!
The letter e is not my favourite!
The letter a is my favourite and it appears in the RDD 3 times
The letter k is not my favourite!
The letter y is not my favourite!
The letter i is not my favourite!
The letter o is my favourite and it appears in the RDD 6 times
The letter m is not my favourite!
The letter c is not my favourite!


favouriteLetters = Vector(a, d, o)
favLetRdd = MapPartitionsRDD[29] at map at <console>:31
joined = MapPartitionsRDD[32] at leftOuterJoin at <console>:34


MapPartitionsRDD[32] at leftOuterJoin at <console>:34

In [139]:
case class Apple(size: Int, color: String)

defined class Apple


In [140]:
val foo: Apple = Apple(1, "red")

foo = Apple(1,red)


Apple(1,red)

In [141]:
foo.size

1

In [142]:
foo.color

red

In [143]:
val Apple(size, color) = foo

size = 1
color = red


red

In [144]:
val bar: List[Apple] = List(foo, foo)

bar = List(Apple(1,red), Apple(1,red))


List(Apple(1,red), Apple(1,red))

In [145]:
Apple(color="red", size=1).productIterator.toList

List(1, red)

In [148]:
Apple(color="red", size=1) == Apple(color="red", size=1)

true

In [149]:
class Foo(i: Int)

defined class Foo


In [151]:
val foo = new Foo(2)

foo = Foo@f76d2b1


Foo@f76d2b1

In [156]:
object Foo2 { 
    val i: Int = 2
    
    def f(i: Int) {
        println(i)
    }
}

defined object Foo2


In [157]:
Foo2.f(1)

1


In [159]:
trait Fruit { 
    val color: String
    
    def getColorUpperCase(): String = { 
        this.color.toUpperCase
    }
}

defined trait Fruit


In [160]:
case class Apple(size: Int, color: String) extends Fruit

defined class Apple


In [161]:
case class Banana(origin: String, color: String) extends Fruit

defined class Banana


In [162]:
val myList: List[Fruit] = List(Apple(1, "green"), Banana("Spain", "yellow"))

myList = List(Apple(1,green), Banana(Spain,yellow))


List(Apple(1,green), Banana(Spain,yellow))

In [166]:
myList.map { 
    case Apple(s, c) => s"Apple: $s, $c"
    case Banana(o, c) => s"Banana: $o, $c"
    case _ => s"Unknown fruit"
}

Name: Unknown Error
Message: <console>:34: error: not found: value Fruit
           case Fruit(c) => s"Unknown fruit, color: ${c}"
                ^

StackTrace: 

In [167]:
Apple(1, "green").getColorUpperCase

GREEN

In [168]:
trait Fruit
trait Round
trait CanBeRed

defined trait Fruit
defined trait Round
defined trait CanBeRed


In [169]:
case class Apple(size: Int) extends Fruit with Round with CanBeRed

defined class Apple


### Выводы
- PairRDD функции - расширенный список функций, доступный для RDD, элементы которых являются кортежем (K, V)
- PairRDD позволяют соединять два RDD по ключу K

### Полезные ссылки:
- [PairRDD API Reference](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.PairRDDFunctions)

## Работа с данными

Для изучения структуры и вычислений RDD проведем анализ датасета [Airport Codes](https://datahub.io/core/airport-codes)  

Метод `sc.textFile` позволяет прочитать файл на локальной, S3 или HDFS совместимой ФС. С помощью данного метода можно читать как обычные файлы, так и директории с файлами, а также архивы с данными.

In [171]:
val rdd = sc.textFile("/tmp/airport-codes.csv")

rdd = /tmp/airport-codes.csv MapPartitionsRDD[36] at textFile at <console>:28


/tmp/airport-codes.csv MapPartitionsRDD[36] at textFile at <console>:28

Выведем первые 3 строки на экран:

In [172]:
rdd.take(3)

Array(ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates, 00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,,00A,"40.07080078125, -74.93360137939453", 00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,,00AA,"38.704022, -101.473911")

In [173]:
rdd.take(3).foreach(println)

ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,,00A,"40.07080078125, -74.93360137939453"
00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,,00AA,"38.704022, -101.473911"


Подготовим `case class` для парсинга данных

In [174]:
case class Airport(
    ident: String,
    `type`: String,
    name: String,
    elevationFt: String,
    continent: String,
    isoCountry: String,
    isoRegion: String,
    municipality: String,
    gpsCode: String,
    iataCode: String,
    localCode: String,
    longitude: String,
    latitude: String
)

defined class Airport


Уберем шапку и ненужные кавычки

In [175]:
val firstElem = rdd.first

val noHeader = rdd.filter(x => x != firstElem).map(x => x.replaceAll("\"", ""))
noHeader.first

firstElem = ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
noHeader = MapPartitionsRDD[38] at map at <console>:30


00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,,00A,40.07080078125, -74.93360137939453

Напишем функцию, которая преобразует `RDD[String] => RDD[Airport]`

In [176]:
def toAirport(data: String): Airport = {
    val airportArr: Array[String] = data.split(",").map(_.trim)
    val Array(
        ident, 
        aType, 
        name, 
        elevationFt, 
        continent, 
        isoCountry, 
        isoRegion, 
        municipality, 
        gpsCode, 
        iataCode, 
        localCode, 
        longitude,
        latitude) = airportArr
    
    Airport(
        ident = ident,
        `type` = aType,
        name = name,
        elevationFt = elevationFt,
        continent = continent,
        isoCountry = isoCountry,
        isoRegion = isoRegion,
        municipality = municipality,
        gpsCode = gpsCode,
        iataCode = iataCode,
        localCode = localCode,
        longitude = longitude,
        latitude = latitude
    )
    
}

toAirport: (data: String)Airport


Выполним преобразование RDD

In [177]:
val airportRdd: RDD[Airport] = noHeader.map(x => toAirport(x))

airportRdd = MapPartitionsRDD[39] at map at <console>:32


MapPartitionsRDD[39] at map at <console>:32

Поскольку любые трансформации являются ленивыми, отсутствие ошибок при выполнении предыдущей ячейки еще не означает, что данная функция отрабатывает корректно на всем датасете. Проверим это с помощью операции `count`:

In [178]:
airportRdd.count

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 1 in stage 47.0 failed 4 times, most recent failure: Lost task 1.3 in stage 47.0 (TID 72, spark-node-1.newprolab.com, executor 2): scala.MatchError: [Ljava.lang.String;@1ca02702 (of class [Ljava.lang.String;)
	at $line524.$read$$iw$$iw$$iw$$iw$$iw$$iw.toAirport(<console>:43)
	at $line525.$read$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:32)
	at $line525.$read$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:32)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at org.apache.spark.util.Utils$.getIteratorSize(Utils.scala:1819)
	at org.apache.spark.rdd.RDD$$anonfun$count$1.apply(RDD.scala:1213)
	at org.apache.spark.rdd.RDD$$anonfun$count$1.apply(RDD.scala:1213)
	at org.apache.spark.SparkContext$$anonfun$runJob$5.apply(SparkContext.scala:2101)
	at org.apache.spark.SparkContext$$anonfun$runJob$5.apply(SparkContext.scala:2101)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultT

Что произошло? `count`, как и любой action, запускает вычисление всех элементов в RDD. Если посмотреть стектрейс, мы увидим причину возникновения ошибки: 

`Caused by: scala.MatchError: [Ljava.lang.String;@42ee14f9 (of class [Ljava.lang.String;)`

Это означает, что размер массива, полученного после операции `split`, меньше количества переменных, которые мы указали в данной операции:

```scala
val Array(
        ident, 
        aType, 
        name, 
        elevationFt, 
        continent, 
        isoCountry, 
        isoRegion, 
        municipality, 
        gpsCode, 
        iataCode, 
        localCode, 
        longitude,
        latitude) = airportArr
```

Изменим код функции `toAirport`, чтобы решить данную проблему. Для простоты будем считать, что если в строке недостаточно элементов, то эту строку следует выкинуть (сделать `None`)

In [180]:
def toAirportOpt(data: String): Option[Airport] = {
    val airportArr: Array[String] = data.split(",", -1)
    
    airportArr match {
        case Array(
            ident, 
            aType, 
            name, 
            elevationFt, 
            continent, 
            isoCountry, 
            isoRegion, 
            municipality, 
            gpsCode, 
            iataCode, 
            localCode, 
            longitude,
            latitude) => {
        
                Some(
                    Airport(
                        ident = ident,
                        `type` = aType,
                        name = name,
                        elevationFt = elevationFt,
                        continent = continent,
                        isoCountry = isoCountry,
                        isoRegion = isoRegion,
                        municipality = municipality,
                        gpsCode = gpsCode,
                        iataCode = iataCode,
                        localCode = localCode,
                        longitude = longitude,
                        latitude = latitude
                        )
                    )
        }
        case _ => Option.empty[Airport]
    }
    
}

toAirportOpt: (data: String)Option[Airport]


Применим новую функцию к RDD:

In [181]:
val airportOptRdd: RDD[Option[Airport]] = noHeader.map(toAirportOpt)

airportOptRdd = MapPartitionsRDD[40] at map at <console>:32


MapPartitionsRDD[40] at map at <console>:32

Проверим корректность выполнения функции на первых трех элементах и на всем датасете:

In [182]:
airportOptRdd.take(3).foreach(println)
airportOptRdd.count

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: ClassNotFound with classloader: scala.reflect.internal.util.ScalaClassLoader$URLClassLoader@524f9610
StackTrace:   at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1925)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1913)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1912)
  at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
  at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
  at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1912)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:948)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:948)
  at scala.Option.fore

In [183]:
val airportRdd: RDD[Airport] = noHeader.flatMap(toAirportOpt)

airportRdd = MapPartitionsRDD[41] at flatMap at <console>:32


lastException: Throwable = null


MapPartitionsRDD[41] at flatMap at <console>:32

In [184]:
airportRdd.count

54944

Добавим корректную обработку числовых типов в наш код:

In [185]:
case class AirportTyped(
    ident: String,
    `type`: String,
    name: String,
    elevationFt: Int,
    continent: String,
    isoCountry: String,
    isoRegion: String,
    municipality: String,
    gpsCode: String,
    iataCode: String,
    localCode: String,
    longitude: Float,
    latitude: Float
)

defined class AirportTyped


In [186]:
def toAirportOptTyped(data: String): Option[AirportTyped] = {
    val airportArr: Array[String] = data.split(",", -1)
    
    airportArr match {
        case Array(
            ident, 
            aType, 
            name, 
            elevationFt, 
            continent, 
            isoCountry, 
            isoRegion, 
            municipality, 
            gpsCode, 
            iataCode, 
            localCode, 
            longitude,
            latitude) => {
        
                Some(
                    AirportTyped(
                        ident = ident,
                        `type` = aType,
                        name = name,
                        elevationFt = elevationFt.toInt,
                        continent = continent,
                        isoCountry = isoCountry,
                        isoRegion = isoRegion,
                        municipality = municipality,
                        gpsCode = gpsCode,
                        iataCode = iataCode,
                        localCode = localCode,
                        longitude = longitude.toFloat,
                        latitude = latitude.toFloat
                        )
                    )
        }
        case _ => Option.empty[AirportTyped]
    }
    
}

toAirportOptTyped: (data: String)Option[AirportTyped]


In [187]:
val airportRddTyped: RDD[AirportTyped] = noHeader.flatMap(toAirportOptTyped)
airportRddTyped.count

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 50.0 failed 4 times, most recent failure: Lost task 0.3 in stage 50.0 (TID 82, spark-node-1.newprolab.com, executor 2): java.lang.NumberFormatException: For input string: ""
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Integer.parseInt(Integer.java:592)
	at java.lang.Integer.parseInt(Integer.java:615)
	at scala.collection.immutable.StringLike$class.toInt(StringLike.scala:273)
	at scala.collection.immutable.StringOps.toInt(StringOps.scala:29)
	at $line555.$read$$iw$$iw$$iw$$iw$$iw$$iw.toAirportOptTyped(<console>:52)
	at $line556.$read$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:32)
	at $line556.$read$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:32)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:441)
	at org.apache.spark.util.Utils$.getIteratorSize(

Теперь у нас новая ошибка:

`java.lang.NumberFormatException: For input string: ""`

Она возникает, когда мы пытаемся превратить пустую строку в Int с помощью метода `.toInt`
Для решения этой задачи мы можем использовать монаду `Try[T]`

In [188]:
import scala.util.Try

case class AirportSafe(
    ident: String,
    `type`: String,
    name: String,
    elevationFt: Option[Int],
    continent: String,
    isoCountry: String,
    isoRegion: String,
    municipality: String,
    gpsCode: String,
    iataCode: String,
    localCode: String,
    longitude: Option[Float],
    latitude: Option[Float]
)

def toAirportOptSafe(data: String): Option[AirportSafe] = {
    val airportArr: Array[String] = data.split(",", -1)
    
    airportArr match {
        case Array(
            ident, 
            aType, 
            name, 
            elevationFt, 
            continent, 
            isoCountry, 
            isoRegion, 
            municipality, 
            gpsCode, 
            iataCode, 
            localCode, 
            longitude,
            latitude) => {
        
                Some(
                    AirportSafe(
                        ident = ident,
                        `type` = aType,
                        name = name,
                        elevationFt = Try(elevationFt.toInt).toOption,
                        continent = continent,
                        isoCountry = isoCountry,
                        isoRegion = isoRegion,
                        municipality = municipality,
                        gpsCode = gpsCode,
                        iataCode = iataCode,
                        localCode = localCode,
                        longitude = Try(longitude.toFloat).toOption,
                        latitude = Try(latitude.toFloat).toOption
                        )
                    )
        }
        case _ => Option.empty[AirportSafe]
    }
    
}

defined class AirportSafe


lastException: Throwable = null
toAirportOptSafe: (data: String)Option[AirportSafe]


Применим данную функцию к нашему датасету:

In [189]:
val airportSafeRdd: RDD[Option[AirportSafe]] = noHeader.map(toAirportOptSafe)

airportSafeRdd = MapPartitionsRDD[43] at map at <console>:33


MapPartitionsRDD[43] at map at <console>:33

Проверим ее применимость:

In [191]:
//airportSafeRdd.take(3).foreach(println)
airportSafeRdd.count

lastException: Throwable = null


55113

`Option[T]` - это удобная монада, которая позволяет работать с отсутствующими данными, избегая исключительных ситуаций и обработки `null`. Одним из ее преимуществ является то, что ее можно рассматривать как коллекцию, что позволяет применить к `RDD[Option[T]]` метод `flatMap`, который вернет RDD[T], убрав все `None` из нашего датасета:

In [194]:
val airportFinal: RDD[AirportSafe] = noHeader.flatMap(toAirportOptSafe)
//airportFinal.take(3).foreach(println)

airportFinal = MapPartitionsRDD[45] at flatMap at <console>:33


MapPartitionsRDD[45] at flatMap at <console>:33

Получим коллекцию, содержащую максимальную высота аэропорта с разбивкой по странам. Для этого первым шагом получим `PairRDD`: `RDD[(K,V)]`, где `K` - это страна, а `V` - высота

In [196]:
val pairAirport = airportFinal.map(x => (x.isoCountry, x.elevationFt))
pairAirport.take(2)

pairAirport = MapPartitionsRDD[47] at map at <console>:31


Array((US,Some(11)), (US,Some(3435)))

Поскольку мы не можем напрямую сравнивать два объекта `Option[T]`, то нам необходимо получить `T`. Будем считать, что аэропорты, где атрибут `elevationFt` принимает значение `None`, необходимо поместить в конец нашего списка. Для этого применим функцию:

In [197]:
val fixedElevation: RDD[(String, Int)] = pairAirport.map {
    case (k, Some(v)) => (k, v)
    case (k, None) => (k, Int.MinValue)
}
fixedElevation.first

fixedElevation = MapPartitionsRDD[48] at map at <console>:29


(US,11)

Теперь нам необходимо применить функцию reduceByKey и получить нужный результат:

In [198]:
import scala.math.max

val result = fixedElevation.reduceByKey { (x, y) => Math.max(x,y) }.collect.sortBy( x => -x._2 )
result.take(10).foreach(println)

(IN,22000)
(PE,14965)
(CN,14472)
(BO,14360)
(CO,13119)
(AR,13000)
(CL,12468)
(US,12442)
(NP,12400)
(TJ,11962)


result = Array((IN,22000), (PE,14965), (CN,14472), (BO,14360), (CO,13119), (AR,13000), (CL,12468), (US,12442), (NP,12400), (TJ,11962), (FR,11647), (CH,10837), (AF,10490), (LS,10400), (KE,10200), (EC,9649), (AQ,9300), (ID,9288), (MX,9121), (BT,9000), (ET,8490), (PG,8400), (KG,8250), (GT,7933), (TZ,7795), (MW,7759), (ER,7661), (AT,7522), (IR,7385), (PK,7316), (MN,7260), (YE,7216), (SA,6858), (BR,6825), (RU,6695), (CD,6562), (OM,6500), (ZA,6464), (TR,6400), (UG,6200), (RW,6102), (NA,6063), (KZ,6051), (MM,6000), (IT,5938), (AO,5778), (BI,5741), (SO,5720), (AU,5689), (HN,5475), (MA,5459), (ZM,5454), (ZW,5370), (CA,5350), (VE,5269), (AM,5000), (PA,5000), (MG,4997), (VN,4937), (KR,4816), (GE,4778), (CR,4650), (CM,4593), (KP,4547), (DZ,4518), (MZ,4505...


Array((IN,22000), (PE,14965), (CN,14472), (BO,14360), (CO,13119), (AR,13000), (CL,12468), (US,12442), (NP,12400), (TJ,11962), (FR,11647), (CH,10837), (AF,10490), (LS,10400), (KE,10200), (EC,9649), (AQ,9300), (ID,9288), (MX,9121), (BT,9000), (ET,8490), (PG,8400), (KG,8250), (GT,7933), (TZ,7795), (MW,7759), (ER,7661), (AT,7522), (IR,7385), (PK,7316), (MN,7260), (YE,7216), (SA,6858), (BR,6825), (RU,6695), (CD,6562), (OM,6500), (ZA,6464), (TR,6400), (UG,6200), (RW,6102), (NA,6063), (KZ,6051), (MM,6000), (IT,5938), (AO,5778), (BI,5741), (SO,5720), (AU,5689), (HN,5475), (MA,5459), (ZM,5454), (ZW,5370), (CA,5350), (VE,5269), (AM,5000), (PA,5000), (MG,4997), (VN,4937), (KR,4816), (GE,4778), (CR,4650), (CM,4593), (KP,4547), (DZ,4518), (MZ,4505...

### Выводы
- RDD API - это низкоуровневый API, который позволяет применять любые функции к распределенным данным
- При использовании RDD API обработка всех исключительных ситуаций лежит на плечах разработчика

После завершения работы не забывайте останавливать `SparkSession`, чтобы освободить ресурсы кластера!

In [ ]:
spark.stop